# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 42 new papers today
          13 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/12 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2208.12274


extracting tarball to tmp_2208.12274...

 done.
Retrieving document from  https://arxiv.org/e-print/2208.12362


extracting tarball to tmp_2208.12362...

...

/tmp/ipykernel_2202/4030337529.py:34: LatexWarning: 2208.12658 did not run properly
not a gzip file
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


Retrieving document from  https://arxiv.org/e-print/2208.12679


extracting tarball to tmp_2208.12679...

 done.


/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3398: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3398: LatexWarning: Found documentclass in tmp_2208.12679/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'can-table.tex' from 'tmp_2208.12679/can-table.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'maser-table.tex' from 'tmp_2208.12679/maser-table.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/

### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.12274-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.12274) | **Indication of a Local Source of Ultra-High-Energy Cosmic Rays in the  Northern Hemisphere**  |
|| Pavlo Plotko, Arjen van Vliet, Xavier Rodrigues, <mark>Walter Winter</mark> |
|*Appeared on*| *2022-08-29*|
|*Comments*| *24 pages, 10 figures, 5 tables*|
|**Abstract**| The Pierre Auger Observatory (PAO) and Telescope Array (TA) collaborations report significant differences in the observed energy spectra of ultra-high-energy cosmic rays (UHECRs) above 30 EeV. In this work, we present a joint fit of TA and PAO data using the rigidity-dependent maximum energy model, and including full marginalization over all relevant parameters. We show that the presence of a local astrophysical source in the Northern Hemisphere, which is only visible by the TA experiment, can reconcile PAO and TA measurements up to the highest energies. We demonstrate that the presence of that local source is favored at the 5.6$\sigma$ level compared to the scenario where both experiments observe the same UHECR flux from a cosmological source distribution. We also quantify that the astrophysical explanation can describe the current data better than a scenario where the differences in the observations are explained by experimental systematics (i.e., energy-dependent shifts). Having tested different mass compositions emitted from the local source, we conclude that the data are best described by a source lying at a distance of about 14 Mpc that emits cosmic rays dominated by the silicon mass group; we also discuss possible source candidates. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.12362-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.12362) | **Laue and Fresnel lenses**  |
|| Enrico Virgilli, <mark>Hubert Halloin</mark>, Gerald Skinner |
|*Appeared on*| *2022-08-29*|
|*Comments*| *43 pages, 17 Figures - accepted Chapter for publication in the Section "Optics and Detectors for Gamma-ray Astrophysics" of the "Handbook of X-ray and Gamma-ray Astrophysics" by Springer - Editors in chief: C. Bambi and A. Santangelo*|
|**Abstract**| The low-energy gamma-ray domain is an important window for the study of the high energy Universe. Here matter can be observed in extreme physical conditions and during powerful explosive events. However, observing gamma-rays from faint sources is extremely challenging with current instrumentation. With techniques used at present collecting more signal requires larger detectors, leading to an increase in instrumental background. For the leap in sensitivity that is required for future gamma-ray missions use must be made of flux concentrating telescopes. Fortunately, gamma-ray optics such as Laue or Fresnel lenses, based on diffraction, make this possible. Laue lenses work with moderate focal lengths (tens to a few hundreds of metres), but provide only rudimentary imaging capabilities. On the other hand, Fresnel lenses offer extremely good imaging, but with a very small field of view and a requirement for focal lengths $\sim$10$^8$ m. This chapter presents the basic concepts of these optics and describes their working principles, their main properties and some feasibility studies already conducted. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.12365-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.12365) | **The Astrodust+PAH Model: A Unified Description of the Extinction,  Emission, and Polarization from Dust in the Diffuse Interstellar Medium**  |
|| <mark>Brandon S. Hensley</mark>, B. T. Draine |
|*Appeared on*| *2022-08-29*|
|*Comments*| *32 pages, 18 figures. Submitted to ApJ. Data available at this https URL*|
|**Abstract**| We present a new model of interstellar dust in which large grains are a single composite material, ``astrodust,'' and nanoparticle-sized grains come in distinct varieties including polycyclic aromatic hydrocarbons (PAHs). We argue that a single-composition model for grains larger than $\sim0.02\,\mu$m most naturally explains the lack of frequency dependence in the far-infrared (FIR) polarization fraction and the characteristic ratio of optical to FIR polarization. We derive a size distribution and alignment function for 1.4:1 oblate astrodust grains that, with PAHs, reproduce the mean wavelength dependence and polarization of Galactic extinction and emission from the diffuse interstellar medium while respecting constraints on solid phase abundances. All model data and Python-based interfaces are made publicly available. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.12556-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.12556) | **Performance of the SRON Ti/Au Transition Edge Sensor X-ray Calorimeters**  |
|| Martin de Wit, et al. -- incl., <mark>Jan-Willem A. den Herder</mark> |
|*Appeared on*| *2022-08-29*|
|*Comments*| *Submitted as a conference proceeding for SPIE Astronomical Telescopes + Instrumentation 2022*|
|**Abstract**| In the early 2030s, ESAs new X-ray observatory, Athena, is scheduled to be launched. It will carry two main instruments, one of which is the X-ray Integral Field Unit (X-IFU), an X-ray imaging spectrometer, which will consist of an array of several thousand transition-edge sensors (TESs) with a proposed energy resolution of 2.5 eV for photon energies up to 7 keV. At SRON we develop the backup TES array based on Ti/Au bilayers with a transition temperature just below 100 mK. In this contribution we will give a broad overview of the properties and capabilities of these state-of-the-art detectors. Over the years we have fabricated and studied a large number of detectors with various geometries, providing us with a good understanding of how to precisely control the properties of our detectors. We are able to accurately vary the most important detector properties, such as the normal resistance, thermal conductance and critical temperature. This allows us to finely tune our detectors to meet the demands of various applications. The detectors have demonstrated excellent energy resolutions of below 1.8 eV for 5.9 keV X-rays. By tuning the properties of the devices, they can be optimally matched to various read-out schemes using both AC and DC biasing. The next step is to increase the size of our TES arrays from our current kilo-pixel arrays towards the full-sized array for X-IFU. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.12569-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.12569) | **FastQSL: a fast computation method for Quasi-Separatrix Layers**  |
|| <mark>Peijin Zhang</mark>, Jun Chen, Rui Liu, Chuanbing Wang |
|*Appeared on*| *2022-08-29*|
|*Comments*| **|
|**Abstract**| Magnetic reconnection preferentially takes place at the intersection of two separatrices or two quasi-separatrix layers, which can be quantified by the squashing factor Q, whose calculation is computationally expensive due to the need to trace as many field lines as possible. We developed a method (FastQSL) optimized for obtaining Q and the twist number in a 3D data cube. FastQSL utilizes the hardware acceleration of the graphic process unit (GPU) and adopts a step-size adaptive scheme for the most computationally intensive part: tracing magnetic field lines. As a result, it achieves a computational efficiency of 4.53 million Q values per second. FastQSL is open source, and user-friendly for data import, export, and visualization. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.12604-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.12604) | **Frequency domain multiplexing readout for large arrays of  transition-edge sensors**  |
|| Davide Vaccaro, et al. -- incl., <mark>Bert-Joost van Leeuwen</mark>, <mark>Jan-Willem A. den Herder</mark> |
|*Appeared on*| *2022-08-29*|
|*Comments*| *Version submitted to Nuclear Instruments and Methods A for the proceedings of the Pisa Meeting 2021 conference*|
|**Abstract**| We report our most recent progress and demonstration of a frequency domain multiplexing (FDM) readout technology for transition-edge sensor (TES) arrays, both of which we have been developing, in the framework of the X-IFU instrument on board the future Athena X-ray telescope. Using Ti/Au TES micro-calorimeters, high-Q LC filters and analog/digital electronics developed at SRON and low-noise two-stage SQUID amplifiers from VTT Finland, we demonstrated feasibility of our FDM readout technology, with the simultaneous readout of 37 pixels with an energy resolution at of 2.23 eV at an energy of 6 keV. We finally outline our plans for further scaling up and improving our technology. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.12679-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.12679) | **Evidence for an interaction between the Galactic Center clouds  M0.10-0.08 and M0.11-0.11**  |
|| Natalie O. Butterfield, et al. -- incl., <mark>Cornelia C. Lang</mark> |
|*Appeared on*| *2022-08-29*|
|*Comments*| *Accepted for publication in the Astrophysical Journal*|
|**Abstract**| We present high-resolution (~2-3"; ~0.1 pc) radio observations of the Galactic center cloud M0.10-0.08 using the Very Large Array at K and Ka band (~25 and 36 GHz). The M0.10-0.08 cloud is located in a complex environment near the Galactic center Radio Arc and the adjacent M0.11-0.11 molecular cloud. From our data, M0.10-0.08 appears to be a compact molecular cloud (~3 pc) that contains multiple compact molecular cores (5+; <0.4 pc). In this study we detect a total of 15 molecular transitions in M0.10-0.08 from the following molecules: NH3, HC3N, CH3OH, HC5N, CH3CN, and OCS. We have identified more than sixty 36 GHz CH3OH masers in M0.10-0.08 with brightness temperatures above 400 K and 31 maser candidates with temperatures between 100-400 K. We conduct a kinematic analysis of the gas using NH3 and detect multiple velocity components towards this region of the Galactic center. The bulk of the gas in this region has a velocity of 51.5 km/s (M0.10-0.08) with a lower velocity wing at 37.6 km/s. We also detect a relatively faint velocity component at 10.6 km/s that we attribute to being an extension of the M0.11-0.11 cloud. Analysis of the gas kinematics, combined with past X-ray fluorescence observations, suggests M0.10-0.08 and M0.11-0.11 are located in the same vicinity of the Galactic center and could be physically interacting. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.12658-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.12658) | **The gas-phase reaction of NH2 with formaldehyde (CH2O) is not a source  of formamide (NH2CHO) in interstellar environments**  |
|| Kevin M. Douglas, et al. -- incl., <mark>Dwayne E. Heard</mark> |
|*Appeared on*| *2022-08-29*|
|*Comments*| *Manuscript, 14 pages, 4 figures. Supporting Information, 8 pages, 2 figures. Accepted for publication in The Astrophysical Journal Letters*|
|**Abstract**| The first experimental study of the low-temperature kinetics of the gas-phase reaction of NH2 with formaldehyde (CH2O) has been performed. This reaction has previously been suggested as a source of formamide (NH2CHO) in interstellar environments. A pulsed Laval nozzle equipped with laser-flash photolysis and laser-induced fluorescence spectroscopy was used to create and monitor the temporal decay of NH2 in the presence of CH2O. No loss of NH2 could be observed via reaction with CH2O and we place an upper-limit on the rate coefficient of <6x10-12 cm3 molecule-1 s-1 at 34K. Ab initio calculations of the potential energy surface were combined with RRKM calculations to predict a rate coefficient of 6.2x10-14 cm3 molecule-1 s-1 at 35K, consistent with the experimental results. The presence of a significant barrier, 18 kJ mol-1, for the formation of formamide as a product, means that only the H-abstraction channel producing NH3 + CHO, in which the transfer of an H-atom can occur by quantum mechanical tunnelling through a 23 kJ mol-1 barrier, is open at low temperatures. These results are in contrast with a recent theoretical study which suggested that the reaction could proceed without a barrier and was therefore a viable route to gas-phase formamide formation. The calculated rate coefficients were used in an astrochemical model which demonstrated that this reaction produces only negligible amounts of gas-phase formamide under interstellar and circumstellar conditions. The reaction of NH2 with CH2O is therefore not an important source of formamide at low temperatures in interstellar environments. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

20  publications in the last 7 days.
	 _build/html/2208.09335.md
	 _build/html/2208.08872.md
	 _build/html/2204.06393.md
	 _build/html/2204.03335.md
	 _build/html/2204.03253.md
	 _build/html/2204.02998.md
	 _build/html/2204.02109.md
	 _build/html/2204.02017.md
	 _build/html/2204.01824.md
	 _build/html/2204.01758.md
	 _build/html/2204.01245.md
	 _build/html/2204.00793.md
	 _build/html/2204.00342.md
	 _build/html/2203.16959.md
	 _build/html/2203.16856.md
	 _build/html/2203.16735.md
	 _build/html/2203.16734.md
	 _build/html/2203.16504.md
	 _build/html/2203.15822.md
	 _build/html/2203.15811.md


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers